#  **ChatGPT-4o**

Identificação de ideação suicida com o conjunto de dados SDCNL

## **Installation**

In [ ]:
# Uninstall any existing openai installations
#!pip uninstall -y openai
# Install the required version of openai
#!pip install openai==0.28
!pip install openai

## **Importing libraries**

In [ ]:
import os
import openai
import pandas as pd
import re
import random
import os
import asyncio
from openai import AsyncOpenAI

## **Loading data**

In [ ]:
url = 'link púbico do Google Drive de acesso ao CSV de teste'
file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id

# read the data
dataset = pd.read_csv(read_url)
dataset.head()

## **ChatGPT model**

In [ ]:
OPENAI_API_KEY="Chave API do ChatGPT"

In [ ]:
client = AsyncOpenAI(
    api_key=OPENAI_API_KEY,
)

## **Classificação das amostras**

In [ ]:
from google.colab import files

def save_respostas(respostas_llm):
  df = pd.DataFrame(respostas_llm)
  df.to_excel(f'respostas.xlsx', index=False)

  files.download(f'respostas.xlsx')

In [ ]:
async def main() -> None:
  respostas_llm = {}

  sentencas = dataset["text"].to_list()
  targets = dataset["classification"].to_list()

  respostas_llm["text"] = sentencas
  respostas_llm["target"] = targets
  respostas_llm['Respostas'] = []

  for sentenca in sentencas:
        response = await client.chat.completions.create(
              model="gpt-4o",
              messages=[{"role": "user", "content": f"Você é um modelo de IA treinado para classificar frases quanto à ideação suicida. Regras: Se a frase indicar ideação suicida, retorne exatamente 1. Se a frase não indicar ideação suicida, retorne exatamente 0. Não adicione explicações ou qualquer outro texto. Somente retorne 0 ou 1. Frase: {sentenca}"}]
        )
        respostas_llm['Respostas'].append(response.choices[0].message.content)

  save_respostas(respostas_llm)

await main()
